In [68]:
import re
import string

lst_translates = dict()
lst_sinonims = list()

In [69]:
# заменить "зе" на "ӕ", но исключения: "дзе"
def replace_ze_to_ae(word):
  exceptions = ["æнгурзела ","бæрæзе", "хормæзелæг"]
  if word in exceptions:
    return word
  ind_ze = word.find("зе")
  while ind_ze >= 0:
    ind_dze = word.find("дзе", ind_ze - 1)
    if not (ind_dze >= 0 and ind_dze + 1 == ind_ze):
      #print(ind_ze, ind_dze)
      word = word[:ind_ze] + ae + word[ind_ze + 2:]
    ind_ze = word.find("зе", ind_ze + 1)
  return word
  

 сегментирование на слова осетинского текста
 1. удалить весь текст в скобках

 2. сегментировать строку по значениям (по номерам)

 3. удалить все пояснения после разделителей <>/

 4. сегментировать на слова по запятой или точек с запятой

 сегментация русской строки

 1. удалить весь текст в скобках

 2. сегментировать строку по значениям (по номерам и "см.")

 3. сегментировать на слова по запятой или точек с запятой

 4. оставить только два слова

в словарь каждое слово по осетински в key в значение 2 русских слова

In [ ]:
ae = "æ"
dict_replace = {"эе":ae,"ае":ae,"еге":ae,"ее":ae,"ге":ae, "гь":"гъ", "кь":"къ", "ть": "тъ"}

print(pattern)
s = "Дзулееи;къзебаер фаеразгенгард кгенын"
new_s = re.sub(pattern, lambda x: dict_replace[x.group()], s)
new_s

эе|ае|еге|ее|ге|гь|кь|ть


'Дзулæи;къзебæр фæразæнгард кæнын'

In [70]:
delim_end = '<|>|/|:|см.'
delim = "[,;.]+"
dict_replace = {"эе":ae,"ае":ae,"еге":ae,"ее":ae,"ге":ae, "гь":"гъ", "кь":"къ", "ть": "тъ"}
pattern_replace = '|'.join(dict_replace)

digits = "|".join([i+"." for i in list(string.digits)])
ae = "æ"

def str_to_oset_words(str_oset):
  str_oset = re.sub(pattern_replace, lambda x: dict_replace[x.group()], str_oset.lower())
  lst_raw = re.split(digits, re.sub(r'\(.+?\)','',str_oset))
  lst_sinon = []
  for r in lst_raw:
    str_begin = re.split(delim_end, r)[0]  # отрезать пояснения
    #print(list(filter(None, re.split(delim, str_begin))))
    words = list(filter(None, [replace_ze_to_ae(i.strip()) for i in filter(None, re.split(delim, str_begin))]))    # поделили на синонимы
    #print(words)
    lst_sinon.extend(words)
  return lst_sinon

def str_to_rus_words(str_rus):
  lst_raw = (re.split(digits, re.sub(r'\(.+?\)','',str_rus.lower())))[0].strip()
  str_begin = re.split(delim_end, lst_raw)[0].strip()  # отрезать пояснения
  words = list(filter(None, [w.strip() for w in list(filter(None, re.split(delim, str_begin)))[:2]]))  # поделили на синонимы
  return " ".join(words)



def add_translate(lst_sin, translate):
  global lst_translates
  for s in lst_sin:
    lst_translates[s] = translate
    #if s in lst_translates:
    #  print("Повторение ключей", s)
    #else:
    #  lst_translates[s] = translate



In [ ]:
s = "Дзулееи; 1.къзебаер фаеразгенгард кгенын"
str_to_oset_words(s)


['дзулæи', 'къæбæр фæразæнгард кæнын']

In [ ]:
# тестирование
s2 = "Аразаег 1. Саразаег, скаензег, сфаелдисаег(чин) 2. Фыссаег."
s2 = "Хауын 1. Фаелдаехын; тъупп каенын (дзург.) 2. Уарынгей: цаеуын / bjkhkjh"
#s2 = "Каесын 1. Аргьауын, нымайын/ тагьд:ныхъуырын/ тагьд-тагьд:зеркгесын < цаестытае ахаессын 2. Аив каесын, аив дзурын"
s1 = "Обратиться 1. Адресоваться, аппелировать; воззвать (уст. и высок) 2. См. повернуться 3. См. сосредоточиться 4. См. превратиться 5. См. прибегнуть"

lst_oset = str_to_oset_words(s2)
print(lst_oset)
rus_translate = str_to_rus_words(s1)
print(rus_translate)
add_translate(lst_oset, rus_translate)
print(lst_translates)
lst_sinonims.append(lst_oset)

[]

{}


In [74]:
import re

path = '/content/drive/MyDrive/Colab Notebooks/NOSU/files/sinonim_dict.txt'

f = open(path, "r")
string_rus = "поехали)"
row = 0
while string_rus:
  #row += 1
  #print(row)
  string_rus = f.readline()
  string_oset = f.readline()
  string_empty = f.readline()
  if string_empty:
    lst_oset = str_to_oset_words(string_oset)
    rus_translate = str_to_rus_words(string_rus)
    #print("rus_translate", rus_translate)
    add_translate(lst_oset, rus_translate)
    lst_sinonims.append(lst_oset)
  else:
    print(row, string_rus)
    print(row, string_oset)
    print("|", string_empty, "|")
    break
print("!!!!!!!")
f.close()

0 Ясновидец, ясновидящий; провидец, прозорливец (книжн.)

0 Зонынджын, дардмаеуынеег, даесны

|  |
!!!!!!!


In [76]:
f = open("oset_sinonims.txt", "w")
f.write(str(lst_sinonims))
f.close()

f = open("oset_translate.txt", "w")
f.write(str(lst_translates))
f.close()

In [75]:
# статистика
# 1531 - словарные статьи
# 4219 - осетинских слова
# 3062 - групп синонимов
print(len(lst_translates))
print(len(lst_sinonims))

4219
3062


In [ ]:
f = open("oset_translate.txt", "r")
s = f.read()
f.close()


In [ ]:
test = open("oset_sinonims.txt", "r")
s = test.read()
test.close()
print(s)
lst = eval(s)
print(lst[0][0])

[['хауын', 'фӕлдӕхын', 'тъупп кӕнын', 'уарынгей']]
хауын
